In [74]:
import tensorflow as tf
import os
from PIL import Image
import numpy
import glob

In [72]:
loaded = tf.saved_model.load("results/1575511630")
loaded_raw = tf.saved_model.load("results/1575561469")

In [73]:
labels = ["up","down","left","right"]
total_correct = 0
total = 0
for counter in range(4):
    path = "./processed3/validate/"+labels[counter]
    x=1
    examples = []
    for filename in glob.glob(os.path.join(path, '*.jpg')):
        if x>1000:
            break
        x+=50    
        image =  Image.open(filename)
        example=numpy.array(image)
        examples.append(example)
    test=tf.convert_to_tensor(examples,tf.float32)
    results = loaded_raw.signatures["predict"](input=test)["classes"]
    unique, counts = numpy.unique(results.numpy(), return_counts=True)
    matrix = dict(zip(unique, counts))
    print(labels[counter])
    print(matrix)
    total_correct += matrix[counter+1]
    total += sum(counts)
    print(matrix[counter+1]/sum(counts))
print(total_correct/total)    

InvalidArgumentError:  Default MaxPoolingOp only supports NHWC on device type CPU
	 [[node resnet_model/max_pooling2d/MaxPool (defined at <ipython-input-72-f810f733f201>:2) ]] [Op:__inference_pruned_29476]

Function call stack:
pruned
